In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier




In [6]:
!pip install xgboost


In [7]:
# Load your data
df = pd.read_csv("telco_churn.csv")

In [8]:
df.head()

customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV StreamingMovies        Contract PaperlessBilling  \
0          No          No              No  Month-to-month              Yes   
1          No          No              No        One year               No   
2          No          No              No  Month-to-month              Yes   
3         Yes          No              No        One year               No   
4          No          No              No  Month-to-month              Yes   

               PaymentMethod MonthlyCharges  TotalCharges Churn  
0           Electronic check          29.85         29.85    No  
1               Mailed check          56.95        1889.5    No  
2               Mailed check          53.85        108.15   Yes  
3  Bank transfer (automatic)          42.30       1840.75    No  
4           Electronic check          70.70        151.65   Yes  

[5 rows x 21 columns]

In [9]:
# Drop customerID if present
df.drop('customerID', axis=1, inplace=True)

In [10]:
# Convert 'TotalCharges' to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)


In [11]:
# Binary conversion
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})

In [12]:
# Encode categorical variables
df = pd.get_dummies(df, drop_first=True)

 # Train-Test Split + SMOTE

In [13]:
X = df.drop("Churn", axis=1)
y = df["Churn"]


In [14]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [15]:
# Handle imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)


 # Train XGBoost Model

In [16]:
model = XGBClassifier(eval_metric='logloss')

model.fit(X_resampled, y_resampled)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [17]:
# Predict probabilities
y_probs = model.predict_proba(X_test)[:, 1]

# Predict with custom threshold
y_pred = (y_probs > 0.4).astype(int)


In [18]:
# Classification report
print("Classification Report:\n")
print(classification_report(y_test, y_pred))

# ROC AUC Score
roc = roc_auc_score(y_test, y_probs)
print(f"ROC AUC Score: {roc:.4f}")


Classification Report:

              precision    recall  f1-score   support

           0       0.86      0.81      0.83      1552
           1       0.54      0.63      0.58       561

    accuracy                           0.76      2113
   macro avg       0.70      0.72      0.71      2113
weighted avg       0.77      0.76      0.76      2113

ROC AUC Score: 0.8016


In [20]:
import joblib

joblib.dump(list(X.columns), "features_names.pkl")


['features_names.pkl']

In [21]:
joblib.dump(model, "churn_model_advanced.pkl")

['churn_model_advanced.pkl']